## Create alarm rule state table

Stores per-stream, per-rule cooldown state (atomic gating).

In [ ]:
import boto3
from botocore.exceptions import ClientError

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "alarm_rule_state"

try:
    table = dynamodb.create_table(
        TableName=TABLE_NAME,
        KeySchema=[
            {"AttributeName": "stream_name", "KeyType": "HASH"},       # Partition key
            {"AttributeName": "rule_id", "KeyType": "RANGE"}        # Sort key
        ],
        AttributeDefinitions=[
            {"AttributeName": "stream_name", "AttributeType": "S"},
            {"AttributeName": "rule_id", "AttributeType": "S"},
        ],
        BillingMode="PAY_PER_REQUEST"
    )
    print("Creating table...")
    table.wait_until_exists()
    print(f"Table '{TABLE_NAME}' created.")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceInUseException":
        print(f"Table '{TABLE_NAME}' already exists.")
    else:
        raise


### Enable TTL on attribute `ttl`
TTL is used for automatic cleanup of cooldown rows.

In [ ]:
import boto3
from botocore.exceptions import ClientError

client = boto3.client("dynamodb", region_name="eu-west-1")
try:
    resp = client.update_time_to_live(
        TableName="alarm_rule_state",
        TimeToLiveSpecification={
            'Enabled': True,
            'AttributeName': 'ttl'
        }
    )
    print("TTL enabled on 'ttl'")
except ClientError as e:
    code = e.response.get('Error', {}).get('Code')
    if code in ("ValidationException", "ResourceInUseException"):
        print(f"TTL already configured or table not in ACTIVE state yet: {code}")
    else:
        raise
